In [16]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [2]:
%load_ext sql
%sql sqlite:///Diamonds

In [19]:
db = %sql select * from Diamond
db = db.DataFrame()
db = db.set_index("id")

 * sqlite:///Diamonds
Done.


In [20]:
db[:5]

,carat,cut,color,clarity,depth,table,x,y,z,price
id,,,,,,,,,,
0,0.30,Premium,D,SI2,62.4,58.0,4.31,4.28,2.68,6.353
1,1.01,Ideal,E,VVS2,62.7,56.0,6.42,6.46,4.04,9.183
2,0.72,Ideal,F,VS2,61.8,59.0,5.71,5.74,3.54,7.983
3,1.08,Very Good,G,SI2,63.2,57.0,6.54,6.50,4.12,8.371
4,0.36,Premium,G,VS1,62.3,59.0,4.50,4.55,2.82,6.588


In [21]:
db.describe()

,carat,depth,table,x,y,z,price
count,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000
mean,0.797576,61.746377,57.451694,5.729163,5.732896,3.537351,7.783822
std,0.475939,1.436796,2.234569,1.125244,1.154799,0.711002,1.016828
min,0.200000,43.000000,44.000000,0.000000,0.000000,0.000000,5.787000
25%,0.400000,61.000000,56.000000,4.710000,4.720000,2.910000,6.851000
50%,0.700000,61.800000,57.000000,5.690000,5.700000,3.520000,7.780000
75%,1.040000,62.500000,59.000000,6.540000,6.540000,4.030000,8.581000
max,5.010000,79.000000,95.000000,10.740000,58.900000,31.800000,9.842000


In [22]:
db.isnull().sum()

carat      0
cut        0
color      0
clarity    0
depth      0
table      0
x          0
y          0
z          0
price      0
dtype: int64

In [23]:
db2 = pd.get_dummies(db, columns=['cut', 'color', 'clarity'])
db2[:10]

,carat,depth,table,x,y,z,price,cut_Fair,cut_Good,cut_Ideal,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
id,,,,,,,,,,,,,,,,,,,,,
0,0.30,62.4,58.0,4.31,4.28,2.68,6.353,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,1.01,62.7,56.0,6.42,6.46,4.04,9.183,0,0,1,...,0,0,0,0,0,0,0,0,0,1
2,0.72,61.8,59.0,5.71,5.74,3.54,7.983,0,0,1,...,0,0,0,0,0,0,0,1,0,0
3,1.08,63.2,57.0,6.54,6.50,4.12,8.371,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0.36,62.3,59.0,4.50,4.55,2.82,6.588,0,0,0,...,0,0,0,0,0,0,1,0,0,0
5,0.53,61.4,57.0,5.18,5.20,3.19,7.496,0,0,1,...,0,0,0,0,0,0,1,0,0,0
6,1.74,62.1,59.0,7.69,7.63,4.76,8.979,0,0,0,...,0,0,0,0,1,0,0,0,0,0
7,0.33,60.0,56.0,4.53,4.56,2.72,6.877,0,0,1,...,0,0,0,0,0,0,0,0,0,1
8,0.32,62.9,56.0,4.36,4.39,2.75,6.554,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [25]:
x = db2.values
columns = db2.columns

In [26]:
min_max_Scaler = MinMaxScaler()
x_scaled = min_max_Scaler.fit_transform(x)
db2 = pd.DataFrame(x_scaled, columns=columns)
db2[:10]

,carat,depth,table,x,y,z,price,cut_Fair,cut_Good,cut_Ideal,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,0.020790,0.538889,0.274510,0.401304,0.072666,0.084277,0.139581,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.168399,0.547222,0.235294,0.597765,0.109677,0.127044,0.837485,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.108108,0.522222,0.294118,0.531657,0.097453,0.111321,0.541554,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.182952,0.561111,0.254902,0.608939,0.110357,0.129560,0.637238,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.033264,0.536111,0.294118,0.418994,0.077250,0.088679,0.197534,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5,0.068607,0.511111,0.254902,0.482309,0.088285,0.100314,0.421455,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6,0.320166,0.530556,0.294118,0.716015,0.129542,0.149686,0.787176,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,0.027027,0.472222,0.235294,0.421788,0.077419,0.085535,0.268804,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8,0.024948,0.552778,0.235294,0.405959,0.074533,0.086478,0.189149,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,0.014553,0.522222,0.254902,0.387337,0.069949,0.080503,0.220962,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [27]:
db2.describe()

,carat,depth,table,x,y,z,price,cut_Fair,cut_Good,cut_Ideal,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
count,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,...,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000
mean,0.124236,0.520733,0.263759,0.533442,0.097333,0.111237,0.492435,0.029712,0.091015,0.401187,...,0.100606,0.052453,0.013670,0.032431,0.241725,0.171079,0.150266,0.228872,0.069386,0.092572
std,0.098948,0.039911,0.043815,0.104771,0.019606,0.022359,0.250759,0.169794,0.287634,0.490145,...,0.300810,0.222942,0.116116,0.177144,0.428134,0.376583,0.357336,0.420112,0.254112,0.289835
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.041580,0.500000,0.235294,0.438547,0.080136,0.091509,0.262392,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.103950,0.522222,0.254902,0.529795,0.096774,0.110692,0.491492,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.174636,0.541667,0.294118,0.608939,0.111036,0.126730,0.689026,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
